In [1]:
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

In [2]:
prom_file_path = "../prom_integrated.parquet"

In [3]:
prom_df = pd.read_parquet(prom_file_path)

In [5]:
print(prom_df[feature_cols].dtypes)

for col in feature_cols:
    for idx, item in enumerate(prom_df[col]):
        if isinstance(item, list) or isinstance(item, dict):
            print(f"Non-scalar type at column {col}, index {idx}")


node_load15                              float64
node_load5                               float64
node_load1                               float64
node_memory_Active_bytes                 float64
nvidia_gpu_power_usage_milliwatts-max    float64
nvidia_gpu_temperature_celsius-max       float64
__index_level_0__                          int64
jobID                                     object
dtype: object
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/wenjun/miniconda3/envs/python_env/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_1728589/1350148012.py", line -1, in <module>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/wenjun/miniconda3/envs/python_env/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/wenjun/miniconda3/envs/python_env/lib/python3.11/site-packages/IPython/core/ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/wenjun/miniconda3/envs/python_env/lib/python3.11/site-packages/IPython/core/ultratb.py", line 1012, i

In [6]:
prom_df.fillna(0, inplace=True)

feature_cols = [col for col in prom_df.columns if col not in ['node', 'timestamp', 'isAnomaly','node_netstat_Udp_InErrors','node_netstat_Tcp_InErrs','node_netstat_Icmp_InErrors','node_filesystem_device_error-sum','finishTasks','jobCount','jobID']]

iforest = IsolationForest(contamination=0.028)

iforest.fit(prom_df[feature_cols])

pred = iforest.predict(prom_df[feature_cols])

pred_labels = [1 if i == -1 else 0 for i in pred]

print(classification_report(prom_df['isAnomaly'], pred_labels))
print(confusion_matrix(prom_df['isAnomaly'], pred_labels))


/home/wenjun/miniconda3/envs/python_env/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


              precision    recall  f1-score   support

           0       0.99      0.99      0.99 101685806
           1       0.67      0.74      0.70   2668388

    accuracy                           0.98 104354194
   macro avg       0.83      0.86      0.85 104354194
weighted avg       0.98      0.98      0.98 104354194

[[100729550    956256]
 [   702731   1965657]]


In [7]:
import joblib
model_filename = 'iforest_slurm_copy_model.joblib'
joblib.dump(iforest, model_filename)

['iforest_slurm_copy_model.joblib']